In [1]:
import pymysql
import pymysql.cursors

In [2]:
conn = pymysql.connect(host='localhost', user = 'root', password='postech', db = 'jawiki', cursorclass=pymysql.cursors.DictCursor)
cursor = conn.cursor()

In [3]:
sql = 'SELECT *'
sql += 'FROM categorylinks A, page B '
sql += 'WHERE A.cl_from = B.page_id'
cursor.execute(sql)
category_link_data = cursor.fetchall()

In [4]:
len(category_link_data)

8224913

In [5]:
import pandas as pd
joined_data = pd.DataFrame(category_link_data, columns = ['cl_from', 'page_title', 'page_namespace', 'page_len','cl_type','cl_to'])
joined_data = joined_data[joined_data.page_title != joined_data.cl_to]

In [6]:
len(joined_data)

8177020

In [7]:
joined_data['page_title'] = joined_data['page_title'].str.decode("utf-8")
joined_data['cl_to'] = joined_data['cl_to'].str.decode("utf-8")

In [8]:
import numpy as np
joined_data['edge'] = list(zip(joined_data.page_title, joined_data.cl_to))
real_data = joined_data[(joined_data.page_namespace == 14)|(joined_data.page_namespace == 0)]

In [9]:
sciecne_node = '科学'

In [10]:
tech_node = '技術'